In [96]:
import pandas as pd

In [97]:
df = pd.read_csv("./data/data.csv")

In [101]:
df.columns

Index(['resume_id', 'contact_email', 'is_male', 'birth_date', 'salary_from',
       'name.1', 'is_married', 'driving_license', 'comment',
       'organization_name', 'working_date_from', 'working_date_to', 'position',
       'comment_len', 'age', 'experience', 'seniority', 'skills',
       'last_3_job_watch', 'wanted_position', 'full_exp_text', 'about'],
      dtype='object')

In [ ]:
df.head()

In [ ]:
df["comment"].replace(r"\n", " ", regex=True, inplace=True)
df.head()

In [ ]:
df.fillna("", inplace=True)
df.head()

In [119]:
df["is_male"].unique()

array([ True, False])

In [ ]:
df["is_male"].replace({True: "Мужчина", False: "Женщина"}, inplace=True)
df.head()

In [121]:
df.columns

Index(['resume_id', 'contact_email', 'is_male', 'birth_date', 'salary_from',
       'name.1', 'is_married', 'driving_license', 'comment',
       'organization_name', 'working_date_from', 'working_date_to', 'position',
       'comment_len', 'age', 'experience', 'seniority', 'skills',
       'last_3_job_watch', 'wanted_position', 'full_exp_text', 'about'],
      dtype='object')

In [150]:
source, target = [], []
last_tmp_text = ""
for res_id in df["resume_id"].unique():

    # source & target
    source_text = ""
    target_text = ""

    # work expirience features
    work_exp = " Опыт работы:"

    org_name = ""
    date_s = ""
    date_f = ""
    pos = ""

    # general features
    gen_feat = ""

    sex = ""
    salary = ""
    bd = ""
    city = ""
    driving_license = ""
    skills = ""
    
    last_view_vacs = ""
    experience = ""
    wanted_pos = ""
    
    # get actual rows and sort
    rows = df[df["resume_id"] == res_id]
    rows = rows.sort_values("working_date_from")

    # work exp counter
    rows_count = 1

    for index, row in rows.iterrows():

        # work exp temp text
        tmp_text = ""
        
        # get general features
        if not all([sex, salary, bd, city, driving_license, skills]):
            sex = row["is_male"]
            salary = row["salary_from"]
            bd = row["birth_date"]
            city = row["name.1"]
            if driving_license == "":
                driving_license = "отсутствует"
            else:
                driving_license = row["driving_license"]
            
            if skills == "":
                skills = "не указано"
            else:
                skills = row["skills"]
        
        # get target text
        if not target_text:
            target_text = row["about"]
        
        if not wanted_pos: wanted_pos = row["wanted_position"]
        if not experience: experience = row["experience"]
        if not last_view_vacs: last_view_vacs = row["last_3_job_watch"]
        
        # get one place work expirience
        org_name = row["organization_name"]
        date_s = row["working_date_from"]
        date_f = row["working_date_to"]
        if not date_f:
            date_f = "по настоящее время"
        pos = row["position"]

        # create one place work exp text
        if pos == "без опыта":
            tmp_text = "Без опыта работы"
        else:
            tmp_text = " " + str(rows_count) + "." + " Место работы: " + str(org_name) + ". Дата начала работы: " + str(date_s) + \
                ". Дата окончания работы: " + str(date_f) + ". Должность: " + str(pos) + "."
        
        if last_tmp_text != tmp_text[3:]:
            work_exp += tmp_text

        last_tmp_text = tmp_text[3:]

        rows_count += 1

    gen_feat_text = "Пол: " + str(sex) + \
        ". Желаемая зарплата: " + str(salary) + " рублей" + \
        ". Желаемая должность: " + str(wanted_pos) + \
        ". Дата рождения: " + str(bd) + \
        ". Город проживания: " + str(city) + \
        ". Водительское удостоверение: " + str(driving_license) + \
        ". Умения/навыки: " + str(skills) + "."
        
    if experience:
        gen_feat_text += " Рабой стаж: " + str(experience) + " лет." 
    
    source_text = gen_feat_text + work_exp
    
    source.append(source_text)
    target.append(target_text)



In [152]:
fin_df = pd.DataFrame(data={"source": source, "target": target})

In [160]:
fin_df.to_csv("./data/sft_dataset.csv")